In [119]:
# All functions use in main programms

from collections import Counter 
import os
from phidl import Device, Layer, LayerSet, make_device
import phidl.geometry as pg
import time as time
import ezdxf
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster




#GDSII


def getverticesGdsii(name,markerlayer=[]):
    
    global GDS
    
    GDS = pg.import_gds(filename = name)
    
      
      
    if len(GDS.layers)>1:
        print('Multiple layers. Type layer and confirm with enter \n')
        for layer in GDS.layers:
                print('#'+str(layer))
        markerlayer=[int(input())]
    else:
        for thelayer in GDS.layers:
            markerlayer=[thelayer]
    
    
    
    GDS.flatten()
    
    GDSLayered = pg.extract(GDS, layers = markerlayer)
    vertices=[]
    for polygon in GDSLayered:
        #print(polygon)
        for vertex in polygon.polygons[0]:
            vertices.append(vertex)
            #print(vertex)
    return np.array(vertices).tolist()
    





##SCON WRITING

def writemarksintoscon(scon,marks,units):
    f=open(scon,'r')
    scontent=f.read()
    if any(regtype in scontent for regtype in ['RL2 ','RL3 ','RL4 ']):
        print('File already has marks, not overwriting')
        return
    sconparts=scontent.split('PC, ')
    if len(marks)==4:
        outputscon=sconparts[0]+'RL4 %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f \n' %(marks[0][0]*units/1000,marks[0][1]*units/1000,marks[1][0]*units/1000,marks[1][1]*units/1000,marks[2][0]*units/1000,marks[2][1]*units/1000,marks[3][0]*units/1000,marks[3][1]*units/1000)
    elif len(marks)==3:
        outputscon=sconparts[0]+'RL3 %.3f, %.3f, %.3f, %.3f, %.3f, %.3f \n' %(marks[0][0]*units/1000,marks[0][1]*units/1000,marks[1][0]*units/1000,marks[1][1]*units/1000,marks[2][0]*units/1000,marks[2][1]*units/1000)    
    elif len(marks)==2:
        outputscon=sconparts[0]+'RL2 %.3f, %.3f, %.3f, %.3f \n' %(marks[0][0]*units/1000,marks[0][1]*units/1000,marks[1][0]*units/1000,marks[1][1]*units/1000) 
    
    for index,i in enumerate(sconparts):
        if index>0:
            outputscon=outputscon+'PC, '+i
        
    outputfilename=scon[:-5]+'Marks'+scon[-5:]
    
    f= open(outputfilename,"w+")
    f.write(outputscon)
    f.close
    
    
def writemarksintotxt(name,marks,units):
    outputscon=''
    for mark in marks:
        print(mark[0],mark[1])
        outputscon=outputscon+'%.3f, %.3f \n' %(mark[0],mark[1])
        
    outputfilename=name[:-5]+'Marks.txt'
    
    f= open(outputfilename,"w+")
    f.write(outputscon)
    f.close
       

        
def findscon(paths):
    allscon=[]
    userindex=1
    for file in paths:
        if '.scon' in file:
            allscon.append(file)
    if len(allscon)==0:
        print('No Scon file in folder')
        return
    elif len(allscon)>1:
        print('Multiple Scon files. Type # of file and confirm with enter \n')
        for index,sconfile in enumerate(allscon):
                print('#'+str(index+1)+' '+sconfile)
        userindex=int(input())
    return(allscon[userindex-1])



## DXF FUNCTIONS


def findallpointsdxf(doc,layers):
    

    msp = doc.modelspace()
    
    points=[]
    
    polylines = msp.query('POLYLINE')
    lwpolys = msp.query('LWPOLYLINE')

    for polyline in polylines:
        if polyline.dxf.layer in layers:
            for location in polyline.points():
                if [location[0],location[1]] not in points:
                    points.append([location[0],location[1]])
    
    for polyline in lwpolys:
        if polyline.dxf.layer in layers:
            for location in polyline.points():
                if [location[0],location[1]] not in points:
                    points.append([location[0],location[1]])
          
    return(points)



def finddxflayer(doc,keyword=''):
    alllayers=[]
    for layer in doc.layers:
        if keyword.lower() in layer.dxf.name.lower():
            alllayers.append(layer.dxf.name)
    
    userindex=1
   
    if len(alllayers)==0:
        print('No layer with keyword found in folder')
        return
    elif len(alllayers)>1:
        print('Multiple layers. Type # of layer and confirm with enter \n')
        for index,layer in enumerate(alllayers):
                print('#'+str(index+1)+' '+layer)
        userindex=int(input())
    return([alllayers[userindex-1]])




## NOT FILEDEPENDENT

#This finds all files that are in the same folder as the script and in 1st level subfolders
#could be expanded iteratively but no priority right now
def getListOfFiles():  
    listOfFile = os.listdir()     
    allFiles = []
    # Iterate over all the entries
    for entry in listOfFile:
        if os.path.isdir(entry):
            subfolder=os.listdir(entry)
            for allsubfiles in subfolder:
                allFiles.append(os.path.join(entry,allsubfiles))
        else:
            allFiles.append(entry)
                
    return allFiles


#This asks/returns files, If there is only one file it autouses it
def finddxforGdsii(paths):
    allvectorgraphic=[]
    userindex=1
    for file in paths:
        if '.dxf' in file or '.gds' in file:
            allvectorgraphic.append(file)
    if len(allvectorgraphic)==0:
        print('No design file in folder')
        return
    elif len(allvectorgraphic)>1:
        print('Multiple design files. Type # of file and confirm with enter \n')
        for index,vecfile in enumerate(allvectorgraphic):
                print('#'+str(index+1)+' '+vecfile)
        userindex=int(input())
    return(allvectorgraphic[userindex-1])


#This is the corefunction, it groups corner points into clusters no larger than max_d in any direction
#then does all parewise centers and picks the most frequent one,
#rounds number to accurcay decimals points (usefull for inaccuracies, i guess rounding depends on units µm or nm)
def findalllignmentmarks(points,max_d=250):   
    accuracy=4
    Z = linkage(points,method='complete',metric='euclidean')                           
    
    
                     
    clusters = fcluster(Z, max_d, criterion='distance')
    
    
    allclusters=[]
    for i in range(1,max(clusters)+1):
        clusterslist=[]
        for j in range(len(clusters)):
            
            if clusters[j]==i:
                clusterslist.append(points[j])
        allclusters.append(clusterslist)
    
        
        
     
    
    
    alignmentmarks = []
    for j in allclusters:
        centers=[]
        for i in j:
            for k in j:
                x=(i[0]+k[0])/2
                y=(i[1]+k[1])/2
                
                x=round(x,accuracy)
                y=round(y,accuracy)
                
                centers.append((x,y))
        
        alignmentmarks.append(most_frequent(centers))
    
    return alignmentmarks


## USED ONLY BY findalllignmentmarks
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 
    
    
    
## Finds the 4 marks farthers away from the center, the center, it uses diamonds moving outwards to detemind the distance  
def findoutermostmarks(marks):   # and sort them
    LL=[0,-100000000]
    LR=[0,-100000000]
    UR=[0,-100000000]
    UL=[0,-100000000]
    for markindex,mark in enumerate(marks):
        LLscore=-mark[0]-mark[1]
        LRscore=-mark[0]+mark[1]
        ULscore=mark[0]-mark[1]
        URscore=mark[0]+mark[1]
        if LLscore>LL[1]:
            LL[1]=LLscore
            LL[0]=markindex
        if LRscore>LR[1]:
            LR[1]=LRscore
            LR[0]=markindex
        if ULscore>UL[1]:
            UL[1]=ULscore
            UL[0]=markindex
        if URscore>UR[1]:
            UR[1]=URscore
            UR[0]=markindex
    return [marks[LL[0]],marks[LR[0]],marks[UL[0]],marks[UR[0]]]

### V2 Upgrades


def getverticesGdsiiv2(name,markerlayer=[]):
    
    global GDS
    
    GDS = pg.import_gds(filename = name)
    
      
      
    if len(GDS.layers)>1:
        print('Multiple layers. Type layer and confirm with enter \n')
        for layer in GDS.layers:
                print('#'+str(layer))
        markerlayer=[int(input())]
    else:
        for thelayer in GDS.layers:
            markerlayer=[thelayer]
    
    
    
    GDS.flatten()
    
    GDSLayered = pg.extract(GDS, layers = markerlayer)
    vertices=[]
    objects=[]
    for polygon in GDSLayered:
        vertices=[]
        for vertex in polygon.polygons[0]:
            
            vertices.append(np.array(vertex).tolist())
            #print(vertex)
        objects.append(vertices)
    return objects
    

def findallpointsdxfv2(doc,layers):
    

    msp = doc.modelspace()
    
    objects=[]
    
    polylines = msp.query('POLYLINE')
    lwpolys = msp.query('LWPOLYLINE')

    for polyline in polylines:
        
        if polyline.dxf.layer in layers:
            vertices=[]
            for location in polyline.points():
                if [location[0],location[1]] not in points:
                    vertices.append([location[0],location[1]])
            objects.append(vertices)
    
    for polyline in lwpolys:
        
        if polyline.dxf.layer in layers:
            vertices=[]
            for location in polyline.points():
                if [location[0],location[1]] not in vertices:
                    vertices.append([location[0],location[1]])
            objects.append(vertices)
          
    return(objects)



def addmeantoboundary(objects):

    means=[]
    for boundary in objects:
        avgx = np.mean(np.array(boundary)[:,0])
        avgy = np.mean(np.array(boundary)[:,1])
        means.append([avgx,avgy])
    
    return means


def findalllignmentmarksv2(means,objects,max_d=250):   
    accuracy=4
    Z = linkage(means,method='complete',metric='euclidean')                           
    
    
                     
    clusters = fcluster(Z, max_d, criterion='distance')
    
    
    allclusters=[]
    for i in range(1,max(clusters)+1):
        clusterslist=[]
        for j in range(len(clusters)):
            
            if clusters[j]==i:
                for coordinate in objects[j]:
                    
                    clusterslist.append(coordinate)
        allclusters.append(clusterslist)
    
        
        
 
    
    
    alignmentmarks = []
    for j in allclusters:
        centers=[]
        for i in j:
            for k in j:
                x=(i[0]+k[0])/2
                y=(i[1]+k[1])/2
                
                x=round(x,accuracy)
                y=round(y,accuracy)
                
                centers.append((x,y))
        
        alignmentmarks.append(most_frequent(centers))
    
    return alignmentmarks


## Experimental

In [ ]:
def writemarksintoscon(scon,marks,units):
    f=open(scon,'r')
    scontent=f.read()
    if any(regtype in scontent for regtype in ['RL2 ','RL3 ','RL4 ']):
        print('File already has marks, not overwriting')
        return
    sconparts=scontent.split('PC, ')
    if len(marks)==4:
        outputscon=sconparts[0]+'RL4 %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f \n' %(marks[0][0]*units/1000,marks[0][1]*units/1000,marks[1][0]*units/1000,marks[1][1]*units/1000,marks[2][0]*units/1000,marks[2][1]*units/1000,marks[3][0]*units/1000,marks[3][1]*units/1000)
    elif len(marks)==3:
        outputscon=sconparts[0]+'RL3 %.3f, %.3f, %.3f, %.3f, %.3f, %.3f \n' %(marks[0][0]*units/1000,marks[0][1]*units/1000,marks[1][0]*units/1000,marks[1][1]*units/1000,marks[2][0]*units/1000,marks[2][1]*units/1000)    
    elif len(marks)==2:
        outputscon=sconparts[0]+'RL2 %.3f, %.3f, %.3f, %.3f \n' %(marks[0][0]*units/1000,marks[0][1]*units/1000,marks[1][0]*units/1000,marks[1][1]*units/1000) 
    
    for index,i in enumerate(sconparts):
        if index>0:
            outputscon=outputscon+'PC, '+i
        
    outputfilename=scon[:-5]+'Marks'+scon[-5:]
    
    f= open(outputfilename,"w+")
    f.write(outputscon)
    f.close
    
    

        
def findscon(paths):
    allscon=[]
    userindex=1
    for file in paths:
        if '.scon' in file:
            allscon.append(file)
    if len(allscon)==0:
        print('No Scon file in folder')
        return
    elif len(allscon)>1:
        print('Multiple Scon files. Type # of file and confirm with enter \n')
        for index,sconfile in enumerate(allscon):
                print('#'+str(index+1)+' '+sconfile)
        userindex=int(input())
    return(allscon[userindex-1])





In [134]:
def getscon(file):         ## Opens a scon file splits it into a list of lines. 

    f=open(file,'r')
    scontent=f.read()
    
    sconparts=scontent.split('\n')
    
    header=[]
    
    for line in sconparts:
        if line[:3]=='PC,':
            break
        header.append(line)
    
    
    return header,sconparts
    


def createwritefieldlist(sconparts,allmarks):   ## Creates and additional line for all writefields containing the 4 closest a
    newsconparts=''                             ## allignmentmarks in the allmarks list
    for index,line in enumerate(sconparts):
        if line[:4]=='PC, ':
            elements=line.split(', ')
            closestmarks = getclosestmarks(allmarks,(float(elements[-2])*1000,float(elements[-1])*1000))
            newsconparts = newsconparts +  createregline(closestmarks,1) + line + '\n'
    return newsconparts
                                           
def createregline(marks,units):   ## writes the alignement marks line,this line needs to be changed for different alignment type
    return 'RL4, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f, %.3f \n' %(marks[0][0]*units/1000,marks[0][1]*units/1000,marks[1][0]*units/1000,marks[1][1]*units/1000,marks[2][0]*units/1000,marks[2][1]*units/1000,marks[3][0]*units/1000,marks[3][1]*units/1000)
    

def getclosestmarks(marks,writefield):     ## finds closest mark to a writefield from a list
    LL=[0,100000000]
    LR=[0,100000000]
    UR=[0,100000000]
    UL=[0,100000000]
    for markindex,mark in enumerate(marks):
        distance=abs((mark[0]-writefield[0]))+abs((mark[1]-writefield[1]))
        direction=(math.copysign(1,(mark[0]-writefield[0])),math.copysign(1,(mark[1]-writefield[1])))
        
        if distance<LL[1] and direction==(-1,-1):
            LL[1]=distance
            LL[0]=markindex
        if distance<LR[1] and direction==(1,-1):
            LR[1]=distance
            LR[0]=markindex
        
        if distance<UL[1] and direction==(-1,1):
            UL[1]=distance
            UL[0]=markindex
        if distance<UR[1] and direction==(1,1):
            UR[1]=distance
            UR[0]=markindex
    return [marks[LL[0]],marks[LR[0]],marks[UL[0]],marks[UR[0]]]
        

def sortscon(scon):            ## Sorts scon file so all writefields with same alignment marks are below each other and the alignment
    sortedscon=[]           ## duplicates are purged
    lines=scon.split('\n')
    for index,line in enumerate(lines):
        if line[:5]=='RL4, ':    ## this line needs to be changed for different alignment type
            if line not in sortedscon:
                sortedscon.append(line)
                sortedscon.append(lines[index+1])
            else:
                sortedscon.insert(sortedscon.index(line), lines[index+1])
                
    return sortedscon



def composescon(header,sortedscon,name):
    outputscon=header[0]+'\n/* adapted by LStampfer-Automarkscript */ \n'
    for line in header[1:]:
        outputscon = outputscon + line + '\n'
    for line in sortedscon: 
        outputscon = outputscon + line + '\n'
    outputscon = outputscon+'!END'
    
    
    outputfilename=scon[:-5]+'Marks'+scon[-5:]
    
    f= open(outputfilename,"w+")
    f.write(outputscon)
    f.close
    
   

## NEW ALGORITHM

In [135]:
from collections import Counter 
import os
from phidl import Device, Layer, LayerSet, make_device
import phidl.geometry as pg
import time as time
import ezdxf
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
import matplotlib.pyplot as plt
import math as math



paths=getListOfFiles()   #or could be done with a folder but i think its best to search in the folder the script is sitting in
                        #so ideally there would be only one file in that folder, then program can autofind it
filename=finddxforGdsii(paths)

if '.gds' in filename:

    objects = getverticesGdsiiv2(filename)

    
    
    
elif '.dxf' in filename:
    
    doc = ezdxf.readfile(filename)

    layer = finddxflayer(doc,keyword='')

    objects =findallpointsdxfv2(doc,layer)
    
    
means = addmeantoboundary(objects)

marks=findalllignmentmarksv2(means, objects,max_d=25)
    
    
writemarksintotxt(filename,marks,1) 


%matplotlib qt
b=np.array(marks)
plt.figure()
plt.plot(b[:,0],b[:,1],lw=0,marker='o')
plt.show()

scon = findscon(paths)
header,sconparts=getscon(scon)
sconparts = createwritefieldlist(sconparts,marks)
sortedscon = sortscon(sconparts)
composescon(header,sortedscon,scon)



if False:
    

    %matplotlib qt
    b=np.array(marks)
    plt.figure()
    plt.plot(b[:,0],b[:,1],lw=0,marker='o')
    plt.show()
    xys=plt.ginput(4)
    
    
    usermarks=[]
    for xy in xys:
        nearestmark=[1e12,1e12]
        for mark in marks:
            if (mark[0]-xy[0])**2 + (mark[1]-xy[1])**2 < (nearestmark[0]-xy[0])**2 + (nearestmark[1]-xy[1])**2:
                nearestmark=mark
        usermarks.append(nearestmark)
        
    writemarksintotxt(filename+'User',marks,1)
print(usermarks)
    

Multiple design files. Type # of file and confirm with enter 

#1 somegdsmarkers.dxf
#2 somegdsmarkers.gds
#3 TestFileDewin.dxf
#4 TestFileDewin.gds
3
Multiple layers. Type # of layer and confirm with enter 

#1 0
#2 TRENCH
#3 RESISTOR
#4 METAL
#5 OPENING_WINDOWS
#6 METAL_MLA
#7 Layer_14
#8 Defpoints
7
67408.0 34492.5
67408.0 34492.5
59908.0 34492.5
59908.0 34492.5
52408.0 34492.5
52408.0 34492.5
59908.0 41992.5
59908.0 41992.5
52408.0 41992.5
52408.0 41992.5
82408.0 41992.5
82408.0 41992.5
74908.0 34492.5
74908.0 34492.5
82408.0 34492.5
82408.0 34492.5
74908.0 49492.5
74908.0 49492.5
82408.0 49492.5
82408.0 49492.5
67408.0 41992.5
67408.0 41992.5
74908.0 41992.5
74908.0 41992.5
81658.0 71992.5
81658.0 71992.5
82408.0 71992.5
82408.0 71992.5
67408.0 71992.5
67408.0 71992.5
68158.0 71992.5
68158.0 71992.5
75658.0 71992.5
75658.0 71992.5
74158.0 71992.5
74158.0 71992.5
74908.0 71992.5
74908.0 71992.5
82408.0 64492.5
82408.0 64492.5
82408.0 56992.5
82408.0 56992.5
67408.0 64492.5
67408.0 